# Týden 5. Škrabání webu 
Škrabání webu (web scraping) je proces shromažďování a analyzování nezpracovaných dat z webu a komunita Pythonu vytvořila několik velmi výkonných nástrojů pro škrabání webu.

V tomto cvičení se dozvíte, jak:

- analyzovat data z webových stránek pomocí řetězcových metod a regulárních výrazů,
- parsovat data webových stránek pomocí parseru HTML,
- pracovat s formuláři a dalšími součástmi webových stránek.

**Pozor**: Některé webové stránky výslovně zakazují uživatelům shromažďovat jejich data pomocí automatizovaných nástrojů, jako jsou ty, které vytvoříte v tomto cvičení. Mnoho opakovaných požadavků na server webové stránky může zpomalovat webovou stránku pro ostatní uživatele a potenciálně přetěžovat server tak, že webová stránka přestane odpovídat úplně. Před použitím svých dovedností v jazyce Python pro scraping webu byste měli vždy zkontrolovat zásady přijatelného používání cílové webové stránky a zjistit, zda přístup k webové stránce pomocí automatizovaných nástrojů není porušením jejích podmínek používání. Z právního hlediska je scraping webu proti vůli webové stránky šedou zónou, a následující techniky mohou být **nezákonné**, pokud jsou použity na webových stránkách, které web scraping zakazují.

## Balíček urllib

Jedním z užitečných balíčků pro web scraping, který najdete ve standardní knihovně Pythonu, je urllib, který obsahuje nástroje pro práci s adresami URL. 

In [1]:
from urllib.request import urlopen

Otevřeme webovou stránku katedry matematiky:

In [2]:
url = "https://mat.fsv.cvut.cz"

Chcete-li otevřít webovou stránku, předejte url příkazu `urlopen()`:

In [3]:
page = urlopen(url)
page

Chcete-li ze stránky získat kód HTML, použijte nejprve metodu `.read()` objektu HTTPResponse, která vrací posloupnost bajtů. Poté použijte metodu `.decode()` k dekódování bajtů do řetězce pomocí UTF-8:

In [4]:
html_bytes = page.read()
html = html_bytes.decode("utf-8")
print(html)

<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <meta http-equiv="cache-control" content="no-cache, must-revalidate">
  <meta http-equiv="pragma" content="no-cache" />
  <link rel="icon" type="image/png" href="znak_icon.png">
  <link rel="shortcut icon" href="znak_icon.ico">
  <link rel="stylesheet" href="2012a.css" type="text/css" media="screen">
  <link rel="stylesheet" href="2012pr.css" type="text/css" media="print">
  <title>ČVUT FSv - K101 Matematika</title>
</head>

<body>
<map name="logo">
  <area href="https://mat.fsv.cvut.cz/" alt="K101" title="K101" shape="rect" coords="4,4,116,43">
  <area href="https://portal.fsv.cvut.cz/katedry/K101" alt="K101" title="K101" shape="rect" coords="4,45,116,55">
  <area href="https://portal.fsv.cvut.cz/" alt="FSv" title="FSv" shape="rect" coords="14,56,106,66">
  <area href="https://portal.cvut.cz/" alt="ČVUT" title="ČVUT" shape="rect" coords="26,67,95,77">
</map>

Pomocí urllib jste k webové stránce přistupovali podobně jako v prohlížeči. Místo vizuálního vykreslování obsahu jste však zdrojový kód získali jako text. Nyní, když máte HTML jako text, můžete z něj získat informace několika různými způsoby.

## Extrakce textu z HTML
Jedním ze způsobů, jak získat informace z HTML stránky, je použití metod řetězců. Například pomocí metody `.find()` můžete v textu HTML vyhledat značky `<title>` a získat název webové stránky.

Pro začátek extrahujte název webové stránky, který jste si vyžádali v předchozím příkladu. Pokud znáte index prvního znaku názvu a index prvního znaku uzavírací značky `</title>`, můžete k extrakci názvu použít řetězcový řez.

Protože funkce `.find()` vrací index prvního výskytu podřetězce, můžete získat index úvodní značky `<title>` předáním řetězce `"<title>"` funkci `.find()`:

In [5]:
title_index = html.find("<title>")
title_index

519

Nechcete však mít index značky `<title>`. Chcete index samotného nadpisu. Chcete-li získat index prvního písmene v nadpisu, můžete k title_index přičíst délku řetězce `"<title>"`:

In [6]:
start_index = title_index + len("<title>")
start_index

526

Nyní získáte index uzavírací značky `</title>` předáním řetězce `"</title>"` příkazu `.find()`:

In [7]:
end_index = html.find("</title>")
end_index

552

Nakonec můžete název extrahovat rozřezáním řetězce html:

In [8]:
title = html[start_index:end_index]
title

'ČVUT FSv - K101 Matematika'

## Balíček re

Regulární výrazy - zkráceně regexy (od angl. regular expressions) - jsou vzory, které můžete použít k vyhledávání textu v řetězci. Python podporuje regulární výrazy prostřednictvím modulu `re` standardní knihovny.

Chcete-li pracovat s regulárními výrazy, musíte nejprve importovat modul `re`:

In [15]:
import re

Regulární výrazy používají k označení různých vzorů speciální znaky zvané metaznaky. Například znak hvězdičky (`*`) označuje nula nebo více výskytů toho, co je těsně před hvězdičkou.

V následujícím příkladu použijete funkci `.findall()` k nalezení libovolného textu v řetězci, který odpovídá zadanému regulárnímu výrazu:

In [16]:
re.findall("ab*c", "ac")

['ac']

Prvním argumentem příkazu `re.findall()` je regulární výraz, který chcete porovnat, a druhým argumentem je řetězec, který se má testovat. Ve výše uvedeném příkladu hledáte vzor `"ab*c"` v řetězci `"ac"`.

Regulární výraz `"ab*c"` odpovídá jakékoli části řetězce, která začíná písmenem `"a"`, končí písmenem `"c"` a mezi nimiž je nula nebo více výskytů písmene `"b"`. `re.findall()` vrátí seznam všech shod. Řetězec `"ac"` odpovídá tomuto vzoru, takže je vrácen v seznamu.

Zde je stejný vzor aplikován na různé řetězce:

In [17]:
print(re.findall("ab*c", "abcd"))
print(re.findall("ab*c", "acc"))
print(re.findall("ab*c", "abcac"))
print(re.findall("ab*c", "abdc"))

['abc']
['ac']
['abc', 'ac']
[]


Všimněte si, že pokud není nalezena žádná shoda, funkce `.findall()` vrátí prázdný seznam.

Při porovnávání vzorů se rozlišují malá a velká písmena. Pokud chcete tento vzor porovnat bez ohledu na velikost písmen, můžete předat třetí argument s hodnotou `re.IGNORECASE`:

In [18]:
re.findall("ab*c", "ABC", re.IGNORECASE)

['ABC']

Tečkou `(.)` můžete v regulárním výrazu označit jakýkoli jednotlivý znak. Například můžete najít všechny řetězce, které obsahují písmena `"a"` a `"c"` oddělená jedním znakem, takto:

In [19]:
re.findall("a.c", "abc")

['abc']

Vzor `.*` uvnitř regulárního výrazu znamená libovolný znak opakovaný libovolný početkrát. Například pomocí `"a.*c"` můžete najít každý podřetězec začínající písmenem `"a"` a končící písmenem `"c"` bez ohledu na to, které písmeno nebo písmena se nacházejí mezi nimi:

In [20]:
re.findall("a.*c", "abbc")

['abbc']

Funkce `re.search()` se často se používá k hledání určitého vzoru uvnitř řetězce. Tato funkce je poněkud složitější než `re.findall()`, protože vrací objekt nazvaný `MatchObject`, který uchovává různé skupiny dat. To proto, že uvnitř jiných shod mohou být shody a funkce `re.search()` vrací všechny možné výsledky.

Podrobnosti o objektu MatchObject nejsou zde podstatné. Prozatím stačí vědět, že volání `.group()` na `MatchObject` vrátí první a nejobsáhlejší výsledek, což je ve většině případů přesně to, co chcete:

In [21]:
match_results = re.search("ab*c", "ABC", re.IGNORECASE)
match_results.group()

'ABC'

V modulu re je ještě jedna funkce, která je užitečná pro analýzu textu. `re.sub()`, což je zkratka pro substitute, umožňuje nahradit text v řetězci, který odpovídá regulárnímu výrazu, novým textem. Chová se podobně jako řetězcová metoda `.replace()`.

Argumenty předávané funkci `re.sub()` jsou regulární výraz, za nímž následuje nahrazovaný text a za ním řetězec. Zde je příklad:

In [22]:
string = "Everything is <replaced> if it's in <tags>."
string = re.sub("<.*>", "ELEPHANTS", string)
string

'Everything is ELEPHANTS.'

Funkce `re.sub()` používá regulární výraz `"<.*>"` k nalezení a nahrazení všeho mezi prvním `<` a posledním `>`, což se rozprostírá od začátku `<replaced>` do konce `<tags>`. Je to proto, že regulární výrazy jazyka Python jsou chamtivé, což znamená, že se snaží najít co nejdelší shodu, pokud jsou použity znaky jako `*`.

Alternativně můžete použít vzor shody `*?`, který funguje stejně jako `*` s tím rozdílem, že odpovídá nejkratšímu možnému řetězci textu:

In [24]:
string = "Everything is <replaced> if it's in <tags>."
string = re.sub("<.*?>", "ELEPHANTS", string)
string

"Everything is ELEPHANTS if it's in ELEPHANTS."

Tentokrát funkce `re.sub()` najde dvě shody, `<replaced>` a `<tags>`, a nahradí obě shody řetězcem `"ELEPHANTS"`.

## Extrakce textu z HTML pomocí regulárních výrazů
Vybaveni těmito znalostmi se nyní pokusme získat nadpis z jiné stránky:

In [10]:
import re
from urllib.request import urlopen

url = "https://mat.fsv.cvut.cz"
page = urlopen(url)
html = page.read().decode("utf-8")

pattern = "<title.*?>.*?</title.*?>"
match_results = re.search(pattern, html, re.IGNORECASE)
title = match_results.group()
title = re.sub("<.*?>", "", title) # Remove HTML tags

print(title)

ČVUT FSv - K101 Matematika


## Použití parseru HTML pro škrabání webu v jazyce Python
Ačkoli jsou regulární výrazy obecně skvělé pro porovnávání vzorů, někdy je jednodušší použít parser HTML, který je výslovně určen pro parsování stránek HTML. Pro tento účel je napsáno mnoho nástrojů v jazyce Python, ale pro začátek je vhodné použít knihovnu [Beautiful Soup]().
```
pip install beautifulsoup4
```

In [11]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = "https://mat.fsv.cvut.cz"
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

Tento program dělá tři věci:

1. Otevře adresu URL https://mat.fsv.cvut.cz pomocí funkce `urlopen()` z modulu `urllib.request`.
2. Přečte HTML ze stránky jako řetězec a přiřadí jej do proměnné `html`.
3. Vytvoří objekt `BeautifulSoup` a přiřadí jej do proměnné `soup`.

Objekt BeautifulSoup přiřazený proměnné soup je vytvořen se dvěma argumenty. Prvním argumentem je HTML, které má být zpracováno, a druhý argument, řetězec "html.parser", říká objektu, který parser má být použit v zákulisí. "html.parser" představuje vestavěný parser jazyka HTML v jazyce Python.

Objekty BeautifulSoup mají metodu `.get_text()`, kterou můžete použít k extrakci veškerého textu z dokumentu a automatickému odstranění všech značek HTML.

In [12]:
print(soup.get_text())












ČVUT FSv - K101 Matematika

















ČVUT
FSv
            - 
            Katedra matematiky,
            Thákurova 7, 166 29 Praha 6
            Tel.: +420 / 224 354 390 


            školní pošta

fakultnírozvrh




            Englishversion









Pro budoucí studenty
•Přijímací zkoušky
 ›Všeobecné informace
 ›Ukázka přijímacího testu
 ›Online testy
•Pomoc s přípravou ke studiu na VŠ
 ›Jarní přípravné kurzy
 ›Přijímací zkoušky „NANEČISTO“
 ›Vstupní vyrovnávací kurz M a DG
  
Pro studenty
•Podmínky vedoucího katedry
•AMOS (přihlašování na on-line testování)
•Bakalářské studium
•Magisterské studium
•Doktorské studium
•Pomoc při studiu matematiky
 ›Centrum Aktivního Učení
  
Pro studenty se zájmem o matematiku

•Modely výuky Matematiky 2
•Univerzita třetího věku
•Volitelné předměty
•Témata bakalářských a diplomových prací
•Rektorysova soutěž
•Vyčichlova soutěž
  
Věda a výzkum
•Dynamical Systems, Prague 2021
•ISCAMI 2019
•CAAS – Theory
•Dynamical Systems, Prague 20

V tomto výstupu je mnoho prázdných řádků. Ty jsou výsledkem znaků nového řádku v textu dokumentu HTML. V případě potřeby je můžete odstranit pomocí řetězcové metody `.replace()`.

In [20]:
print(soup.get_text().replace('\n\n\n', ''))



ČVUT FSv - K101 MatematikaČVUT
FSv
            - 
            Katedra matematiky,
            Thákurova 7, 166 29 Praha 6
            Tel.: +420 / 224 354 390 


            školní pošta

fakultnírozvrh

            Englishversion
Pro budoucí studenty
•Přijímací zkoušky
 ›Všeobecné informace
 ›Ukázka přijímacího testu
 ›Online testy
•Pomoc s přípravou ke studiu na VŠ
 ›Jarní přípravné kurzy
 ›Přijímací zkoušky „NANEČISTO“
 ›Vstupní vyrovnávací kurz M a DG
  
Pro studenty
•Podmínky vedoucího katedry
•AMOS (přihlašování na on-line testování)
•Bakalářské studium
•Magisterské studium
•Doktorské studium
•Pomoc při studiu matematiky
 ›Centrum Aktivního Učení
  
Pro studenty se zájmem o matematiku

•Modely výuky Matematiky 2
•Univerzita třetího věku
•Volitelné předměty
•Témata bakalářských a diplomových prací
•Rektorysova soutěž
•Vyčichlova soutěž
  
Věda a výzkum
•Dynamical Systems, Prague 2021
•ISCAMI 2019
•CAAS – Theory
•Dynamical Systems, Prague 2015
•XXXVIII. Summer Symposium in Real A


Často potřebujete z dokumentu HTML získat pouze konkrétní text. Pomocí Beautiful Soup nejprve extrahovat text a poté použít řetězcovou metodu `.find()` je někdy jednodušší než práce s regulárními výrazy.

Jindy jsou však prvky, které upozorňují na data, která chcete načíst, samotné značky HTML. Možná budete chtít například získat adresy URL všech obrázků na stránce. Tyto odkazy jsou obsaženy v atributu src HTML tagů `<img>`.

V tomto případě můžete pomocí `find_all()` vrátit seznam všech instancí této konkrétní značky:

In [21]:
soup.find_all("img")

[<img alt="ČVUT" border="0" height="81" src="logo_cvut_white.png" title="ČVUT" width="106"/>,
 <img alt="Webové rozhraní poštovního serveru" border="0" height="48" src="Mail2.png" style="margin: 0px 3px 6px 3px;" title="Webové rozhraní poštovního serveru" width="72"/>,
 <img alt="english version" border="0" height="25" src="uk_small.png" style="margin: 3px 3px 12px 3px;" width="50"/>,
 <img align="right" alt="Logo" border="0" height="81" src="znak_small.png" usemap="#logo" width="120"/>]

Tím se vrátí seznam všech značek `<img>` v dokumentu HTML. Objekty v seznamu vypadají, jako by to mohly být řetězce představující značky, ale ve skutečnosti jsou to instance objektu `Tag`, který poskytuje Beautiful Soup. Objekty značek poskytují jednoduché rozhraní pro práci s informacemi, které obsahují.

Můžete to trochu prozkoumat tak, že nejprve rozbalíte objekty značek ze seznamu:

In [24]:
image1, image2, image3, image4 = soup.find_all("img")

Každý objekt `Tag` má vlastnost `.name`, která vrací řetězec obsahující typ značky HTML:

In [25]:
image1.name

'img'

K atributům HTML objektu Tag můžete přistupovat tak, že jejich názvy vložíte do hranatých závorek, stejně jako by atributy byly klíče ve slovníku.

Například značka `<img alt="ČVUT" border="0" height="81" src="logo_cvut_white.png" title="ČVUT" width="106"/>` má 5 atributů, např. `src`, s hodnotou `logo_cvut_white.png"`. 

Chcete-li získat zdroj obrázků na stránce, získáte přístup k atributu src pomocí výše uvedeného slovníkového zápisu:

In [27]:
print(image1["src"])
print(image2["src"])
print(image3["src"])
print(image4["src"])

logo_cvut_white.png
Mail2.png
uk_small.png
znak_small.png


K určitým značkám v dokumentech HTML lze přistupovat pomocí vlastností objektu `Tag`. Chcete-li například v dokumentu získat značku `<title>`, můžete použít vlastnost `.title`:

In [28]:
soup.title

<title>ČVUT FSv - K101 Matematika</title>

Můžete také načíst pouze řetězec mezi tagy `title` pomocí vlastnosti `.string` objektu `Tag`:

In [29]:
soup.title.string

'ČVUT FSv - K101 Matematika'

Jednou z funkcí Beautiful Soup je schopnost vyhledávat konkrétní druhy značek, jejichž atributy odpovídají určitým hodnotám. Pokud například chcete najít všechny značky `<img>`, které mají atribut src rovný hodnotě `logo_cvut_white.png`, můžete pro `.find_all()` zadat následující další argument:

In [30]:
soup.find_all("img", src="logo_cvut_white.png")

[<img alt="ČVUT" border="0" height="81" src="logo_cvut_white.png" title="ČVUT" width="106"/>]

Tento příklad je poněkud libovolný a užitečnost této techniky nemusí být z příkladu zřejmá. Pokud strávíte nějaký čas procházením různých webů a prohlížením jejich zdrojů stránek, pak si všimnete, že mnoho webů má extrémně komplikované HTML struktury.

Když stahujete data z webů pomocí Pythonu, často vás zajímají konkrétní části stránky. Když strávíte nějaký čas prohlížením dokumentu HTML, můžete identifikovat značky s jedinečnými atributy, které můžete použít k extrahování potřebných dat.

Potom místo toho, abyste se spoléhali na komplikované regulární výrazy nebo používali `.find()` k prohledávání dokumentu, můžete přímo přistupovat ke konkrétní značce, která vás zajímá, a extrahovat data, která potřebujete.

## Práce s formuláři

Modul urllib, se kterým jste dosud pracovali v tomto cvičení, se dobře hodí pro vyžádání obsahu webové stránky. Někdy však potřebujete interakci s webovou stránkou, abyste získali požadovaný obsah. Například budete možná muset odeslat formulář nebo kliknout na tlačítko pro zobrazení skrytého obsahu.

Standardní knihovna Pythonu neposkytuje vestavěné prostředky pro interaktivní práci s webovými stránkami, ale z PyPI je k dispozici mnoho balíčků třetích stran. Mezi nimi je [MechanicalSoup](https://mechanicalsoup.readthedocs.io/en/stable/), oblíbený a relativně jednoduchý balíček k použití.

```
python -m pip install MechanicalSoup
```

MechanicalSoup v podstatě nainstaluje to, co je známé jako bezhlavý prohlížeč (headless browser), což je webový prohlížeč bez grafického uživatelského rozhraní. Tento prohlížeč je ovládán programově pomocí Pythonu.

Objekty `Browser` představují webový prohlížeč bez hlavy.

In [39]:
import mechanicalsoup
browser = mechanicalsoup.Browser()

Můžete je použít k vyžádání stránky z internetu předáním adresy URL jejich metodě `.get()`:

In [41]:
url = "https://usermap.cvut.cz/login"
page = browser.get(url)

`page` je objekt `Response`, který ukládá odpověď z požadavku na URL z prohlížeče:

In [42]:
page

<Response [200]>

Číslo `200` představuje stavový kód vrácený požadavkem. Stavový kód `200` znamená, že požadavek byl úspěšný. U neúspěšného požadavku se může zobrazit stavový kód `404`, pokud adresa URL neexistuje, nebo `500`, pokud při zadávání požadavku došlo k chybě serveru.

MechanicalSoup používá Beautiful Soup k analýze HTML z požadavku a stránka má atribut `.soup`, který představuje objekt BeautifulSoup:

In [36]:
type(page.soup)

bs4.BeautifulSoup

HTML můžete zobrazit kontrolou atributu .soup:

In [43]:
page.soup

<!DOCTYPE html>
<html lang="cs">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="5a03c153-507c-4cc7-8de6-7070d5024d28" name="_csrf"/>
<meta content="X-CSRF-TOKEN" name="_csrf_header"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="ČVUT-VIC, Ing. Petr Karel" name="author"/>
<meta content="UserMap - systém pro evidenci a správu uživatelů v rámci IS ČVUT" name="description"/>
<title>Přihlašovací dialog | Usermap</title>
<link href="/assets/images/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/assets/images/favicon-96x96.png" rel="icon" sizes="96x96" type="image/png"/>
<link href="/assets/images/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/assets/base.bundle.css" rel="stylesheet"/>
<script>
            window.APP_CONTEXT = '/';
            window.APP_LANG = 'cs';
         

Všimněte si, že tato stránka obsahuje `<form>` s prvky `<input>` pro uživatelské jméno a heslo.

Důležitou částí HTML kódu je přihlašovací formulář – tedy vše uvnitř značek `<form>`. `<form>` na této stránce má atribut name nastaven na přihlášení. Tento formulář obsahuje dva prvky `<input>`, jeden s názvem `username` a druhý s názvem `password`. Třetím prvkem `<input>` je tlačítko `Přihlásit`.

Nyní, když znáte základní strukturu přihlašovacího formuláře, stejně jako přihlašovací údaje potřebné k přihlášení, podívejte se na program, který formulář vyplní a odešle.

In [55]:
# Initialize the browser
browser = mechanicalsoup.Browser()
url = "https://usermap.cvut.cz/login" # replace with the URL of the page containing the form
login_page = browser.get(url)
login_html = login_page.soup

# Select the form using BeautifulSoup's select method
form = login_html.select("form")[0]

# Fill out the username and password fields
form.select("input[name=username]")[0]['value'] = 'username'  # replace with your KOS username
form.select("input[name=password]")[0]['value'] = 'pwd'  # replace with your KOS password

# Submit the form
response = browser.submit(form, login_page.url)

https://usermap.cvut.cz/search


Chcete-li potvrdit, že jste se úspěšně přihlásili, zadejte do interaktivního okna následující:

In [56]:
print(response.url)

https://usermap.cvut.cz/search


Nyní rozeberte výše uvedený příklad:

1. Vytvoříte instanci prohlížeče a použijete ji k vyžádání adresy URL https://usermap.cvut.cz/login. HTML obsah stránky přiřadíte proměnné login_html pomocí vlastnosti `.soup`.

2. `login_html.select("form")` vrátí seznam všech prvků `<form>` na stránce. Protože stránka obsahuje pouze jeden prvek `<form>`, můžete k formuláři přistupovat načtením prvku na indexu 0 seznamu. Pokud je na stránce pouze jeden formulář, můžete také použít `login_html.form`. Další dva řádky vyberou vstupy uživatelského jména a hesla a nastaví jejich hodnotu na „username“ a „pwd“.

3. Formulář odešlete pomocí `browser.submit()`. Všimněte si, že této metodě předáváte dva argumenty, objekt formuláře a URL stránky login_page, ke které přistupujete přes `login_page.url`.

**Poznámka**: Hackeři mohou používat automatizované programy, jako je ten výše, k hrubému vynucení přihlášení rychlým zkoušením mnoha různých uživatelských jmen a hesel, dokud nenajdou funkční kombinaci. 
Kromě toho, že je to vysoce nelegální, téměř všechny webové stránky vás v dnešní době zablokují a nahlásí vaši IP adresu, pokud uvidí, že děláte příliš mnoho neúspěšných požadavků, takže to nezkoušejte!

## Získaní odkazů

Nyní, když se vrátíme na hlavní stránku katedry, zkusíme získat adresu URL pro každý odkaz na pracovníka katedry.

Chcete-li to provést, použijte znovu `.select()`, tentokrát předáním řetězce `"a"` k výběru všech kotevních prvků `<a>` na stránce:

In [68]:
url = "https://usermap.cvut.cz/login"
page = browser.get(url)

# Select the table with the given attributes
table = soup.select_one('table.tab_staff[cellpadding="0"][cellspacing="0"]')

# If the table exists, extract all the links
if table:
    links = table.select('a')
else:
    print("Table not found.")
#links = page.soup.select("a")

Nyní můžete iterovat každý odkaz a vytisknout atribut `href`:

In [72]:
for link in links:
    address = link["href"]
    text = link.text
    print(f"{text}: {address}")

doc. RNDr. Jozef Bobok, CSc.: zamestnanci/detail/1731803
doc. RNDr. Jan Chleboun, CSc.: zamestnanci/detail/1717481
doc. RNDr. Pavel Krejčí, CSc.: zamestnanci/detail/1297836
RNDr. Iva Malechová, CSc.: zamestnanci/detail/1728439
Lucie Hančlová: zamestnanci/detail/1375439
prof. RNDr. Daniela Jarušková, CSc.: zamestnanci/detail/1731794
MSc. Mohammad Al Janaideh, Ph.D.: zamestnanci/detail/1219592
doc. RNDr. Jozef Bobok, CSc.: zamestnanci/detail/1731803
doc. RNDr. Jan Chleboun, CSc.: zamestnanci/detail/1717481
doc. RNDr. Pavel Krejčí, CSc.: zamestnanci/detail/1297836
doc. RNDr. Petr Kučera, CSc.: zamestnanci/detail/1731785
doc. RNDr. Petr Mayer, Dr.: zamestnanci/detail/1727192
doc. RNDr. Aleš Nekvinda, CSc.: zamestnanci/detail/1731779
doc. RNDr. Ivana Pultarová, Ph.D.: zamestnanci/detail/1731776
doc. RNDr. Zdeněk Skalák, CSc.: zamestnanci/detail/1728447
doc. RNDr. Ondřej Zindulka, CSc.: zamestnanci/detail/1731754
Ing. Michal Beneš, Ph.D.: zamestnanci/detail/1491754
Mgr. Milan Bořík, Ph.D.: z

Adresy URL obsažené v každém atributu href jsou relativní adresy URL, které nejsou příliš užitečné, pokud k nim chcete později přejít pomocí MechanicalSoup. Pokud náhodou znáte úplnou adresu URL, můžete přiřadit část potřebnou k vytvoření úplné adresy URL.

V tomto případě je základní URL pouze `https://mat.fsv.cvut.cz`. Poté můžete zřetězit základní adresu URL s relativními adresami URL nalezenými v atributu `src`:

In [73]:
base_url = "https://mat.fsv.cvut.cz/"
for link in links:
    address = base_url + link["href"]
    text = link.text
    print(f"{text}: {address}")

doc. RNDr. Jozef Bobok, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1731803
doc. RNDr. Jan Chleboun, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1717481
doc. RNDr. Pavel Krejčí, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1297836
RNDr. Iva Malechová, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1728439
Lucie Hančlová: https://mat.fsv.cvut.cz/zamestnanci/detail/1375439
prof. RNDr. Daniela Jarušková, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1731794
MSc. Mohammad Al Janaideh, Ph.D.: https://mat.fsv.cvut.cz/zamestnanci/detail/1219592
doc. RNDr. Jozef Bobok, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1731803
doc. RNDr. Jan Chleboun, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1717481
doc. RNDr. Pavel Krejčí, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1297836
doc. RNDr. Petr Kučera, CSc.: https://mat.fsv.cvut.cz/zamestnanci/detail/1731785
doc. RNDr. Petr Mayer, Dr.: https://mat.fsv.cvut.cz/zamestnanci/detail/1727192
doc. RNDr. Aleš Nekvinda, CSc.: 

## Komunikace s webovými stránkami v reálném čase
ěkdy chcete mít možnost získat data v reálném čase z webové stránky, která nabízí neustále aktualizované informace. Otevřete svůj prohlížeč a přejděte na adresu URL https://www.random.org/dice:

Tato stránka simuluje hod šestistěnnou kostkou a aktualizuje výsledek pokaždé, když obnovíte prohlížeč. Níže napíšete program, který opakovaně seškrábe stránku pro nový výsledek.

První věc, kterou musíte udělat, je určit, který prvek na stránce obsahuje výsledek hodu kostkou. Udělejte to nyní kliknutím pravým tlačítkem kdekoli na stránce a výběrem `Zobrazit zdroj stránky`. O něco více než v polovině kódu HTML je značka `<h2>`, která vypadá takto:

```html
<p>You rolled 2 dice:</p>
<p>
<img src="dice3.png" alt="3" />
<img src="dice5.png" alt="5" />
</p>
<p>Timestamp: 2023-08-30 14:46:20 UTC</p>
```

In [75]:
import mechanicalsoup
import time

def get_dice_roll():
    url = "https://www.random.org/dice/?num=2"

    # Create a browser instance
    browser = mechanicalsoup.StatefulBrowser()

    # Navigate to the page
    response = browser.open(url)

    # Parse the page and find the dice images
    dice_images = browser.page.find_all("img", alt=True)

    # Extract the dice values from the image alt attributes
    dice_values = [int(img['alt'][0]) for img in dice_images]

    # Close the browser
    browser.close()

    return dice_values

# Roll the dice 3 times with 10-second intervals
for i in range(3):
    dice_values = get_dice_roll()
    print(f"Roll {i+1}: {dice_values}")
    if i < 2:  # We don't want to wait after the last roll
        time.sleep(10)

Roll 1: [6, 1]
Roll 2: [1, 2]
Roll 3: [5, 6]


Tento příklad používá metodu `.find_all()` objektu BeautifulSoup k nalezení prvku s `img`.

Chcete-li pravidelně získávat nový výsledek, budete muset vytvořit smyčku, která načte stránku v každém kroku. Takže vše pod řádkem `browser = mechanicsoup.Browser()` ve výše uvedeném kódu musí jít do těla smyčky.

V tomto příkladu chcete čtyři hody kostkou v desetisekundových intervalech. Chcete-li to provést, musí poslední řádek vašeho kódu říci Pythonu, aby pozastavil běh na deset sekund. Můžete to udělat pomocí `.sleep()` z časového modulu Pythonu. Metoda `.sleep()` přebírá jeden argument, který představuje dobu spánku v sekundách.

Zde je příklad, který ilustruje, jak funkce `sleep()` funguje:

In [76]:
print("I'm about to wait for five seconds...")
time.sleep(5)
print("Done waiting!")

I'm about to wait for five seconds...
Done waiting!


Pomocí technik, jako je tato, můžete získávat data z webových stránek, které pravidelně aktualizují svá data. Měli byste si však být vědomi toho, že žádost o stránku několikrát v rychlém sledu může být považována za podezřelé nebo dokonce škodlivé použití webové stránky.

**Důležité**: Většina webových stránek publikuje dokument Podmínky použití. Často na něj najdete odkaz v patičce webu.

Než se pokusíte získat data z webové stránky, vždy si přečtěte tento dokument. Pokud nemůžete najít podmínky použití, zkuste kontaktovat vlastníka webu a zeptat se ho, zda má nějaké zásady týkající se objemu požadavků.

Nedodržení podmínek použití může mít za následek zablokování vaší IP adresy, takže buďte opatrní!

Je dokonce možné zřítit server s nadměrným počtem požadavků, takže si dokážete představit, že mnoho webových stránek má obavy z množství požadavků na svůj server! Při odesílání více žádostí na webovou stránku vždy zkontrolujte Podmínky použití a buďte ohleduplní.